In [1]:
import os, inspect, sys
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
import PIL
import math
from tqdm import tqdm
from torch import nn
from torch.nn import functional as F
from dataset import NormalizedDataset, IncompleteDataset, MinstWrapper, get_stats, get_loaders
from classification import Conv_Net, CNNClassifier
import pickle
from skimage.morphology import dilation

In [3]:
white = 255
black = 0

def inverse_color(img):
    return PIL.Image.eval(img, lambda val: 255 - val)

def fix_background_color_bug(img):
    colors = sorted(img.getcolors(), key=lambda pair: pair[0], reverse=True)
    replace_color = colors[0][1]
    remove_color = colors[2][1] if colors[2][1] < colors[1][1] else colors[1][1]

    data = np.array(img)
    data[data == remove_color] = replace_color
    return PIL.Image.fromarray(data)

def remove_background(img):
    return PIL.Image.eval(img, lambda val: 0 if val < (256/2) else val)

def to_binary(img):
    return PIL.Image.eval(img, lambda val: 255 if val < (256/2) else 0)

def to_dilated(img):
    return PIL.Image.fromarray(dilation(np.array(img)))

def get(name):
    try:
        with open('{}.pickle'.format(name), 'rb') as handle:
            return pickle.load(handle)
    except FileNotFoundError:
        return {}

In [4]:
binary_image = False

## Rotation, Brightness and Resizing
random_transforms = [
    transforms.RandomRotation(360, fill=black),
    transforms.RandomAffine(0, shear=15, scale=(0.8, 0.95), translate=(.03, .03)),
]

format_transforms = [transforms.Lambda(to_dilated)]

if not binary_image:
    random_transforms.append(transforms.ColorJitter(brightness=(0.9, 1), contrast=(0.7, 1)))
else:
    format_transforms.append(transforms.Lambda(to_binary))

all_transforms = transforms.Compose(random_transforms + format_transforms)

video_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Lambda(inverse_color),
    transforms.Lambda(remove_background),
    transforms.Resize((28, 28)),
    transforms.Compose(format_transforms),
])

minst_tensor_transform = transforms.Compose([
    all_transforms,
    transforms.ToTensor(),
])

video_tensor_transform = transforms.Compose([
    video_transform,
    transforms.ToTensor(),
])

In [5]:
# Minst Dataset
minst_dataset = MinstWrapper(datasets.MNIST("../", transform=minst_tensor_transform, download=True))

# Show example image
datasets.MNIST("../", transform=all_transforms, download=True)[0][0]

In [6]:
minst_mean, minst_std = get_stats(minst_dataset)
print(minst_mean, minst_std)
minst_dataset = NormalizedDataset(minst_dataset)

0.15896999835968018 0.2910420000553131


In [7]:
# Creating train and test loader from MINST
train_loader, test_loader = get_loaders(minst_dataset, validation_split=0.2)
len(train_loader), len(test_loader)

(480, 120)

In [8]:
# Importing test set
images = [x[0] for x in get("../digits")]
labels = ["3", "2", "7", "2"]

In [13]:
%%time

model = Conv_Net(nb_hidden=100, nb_conv3=128, nb_out=10)

losses = []
test_losses = []

# Defining the optimizer for GD
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr = lr) 

# Defining the criterion to calculate loss
criterion = nn.CrossEntropyLoss()

# Train the model
nb_epochs = 200
best_acc = 0


# Temporary classifier for testing
classifier = CNNClassifier("digits", path="../", minst_binary=binary_image)
classifier_with_median = CNNClassifier("digits", path="../", minst_binary=binary_image, with_median_filter=True)

for e in range(nb_epochs):
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    model.train()
    for train_input, train_target in train_loader:

        # Model computations
        output = model(train_input)
        loss = criterion(output, train_target) 
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        
        tmp_acc = 0
        for tensor, target in zip(output, train_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
                
        tmp_acc /= len(output)
        acc += tmp_acc 
        
    acc /= len(train_loader)
        
    should_print = e%1 == 0
    if should_print:
        print('%dth epoch, train acc: %f' % (e+1, acc), end="")
    losses.append(loss)
    
    # Train the input dataset by dividing it into mini_batch_size small datasets
    acc = 0
    model.eval()
    for test_input, test_target in test_loader:
        
        output = model(test_input)
        loss = criterion(output, test_target) 
        
        tmp_acc = 0
        for tensor, target in zip(output, test_target):
            _, index = tensor.max(0)
            if index == target:
                tmp_acc += 1
#             elif should_print:
#                 pass
#                 print(", {} != {}".format(index, target), end="")
                
        tmp_acc /= len(output)
        acc += tmp_acc 
                
    acc /= len(test_loader)
    print(", val acc: %f" % (acc), end="")
    
    test_losses.append(loss)
    
    test_acc = 0
    test_acc_with_median = 0
    classifier.model = model
    classifier_with_median.model = model
    for (img, label) in zip(images, labels):
        prediction = classifier.predict(img)
        if prediction == label:
            test_acc += 1
            
        prediction = classifier_with_median.predict(img)
        if prediction == label:
            test_acc_with_median += 1

    test_acc /= len(images)
    test_acc_with_median /= len(images)
    print(", test acc: %f, test acc (with median): %f" % (test_acc, test_acc_with_median))
    
    if test_acc_with_median >= best_acc:
        best_acc = test_acc_with_median
        torch.save(model.state_dict(), "../digit_model_with_median")

1th epoch, train acc: 0.505146, val acc: 0.712083, test acc: 0.750000, test acc (with median): 0.750000
2th epoch, train acc: 0.757562, val acc: 0.819167, test acc: 0.250000, test acc (with median): 0.500000
3th epoch, train acc: 0.818583, val acc: 0.851167, test acc: 0.000000, test acc (with median): 0.250000
4th epoch, train acc: 0.849396, val acc: 0.865833, test acc: 0.750000, test acc (with median): 0.750000
5th epoch, train acc: 0.862167, val acc: 0.871250, test acc: 0.500000, test acc (with median): 0.500000
6th epoch, train acc: 0.872813, val acc: 0.885917, test acc: 0.750000, test acc (with median): 0.500000
7th epoch, train acc: 0.880708, val acc: 0.891167, test acc: 0.750000, test acc (with median): 0.750000
8th epoch, train acc: 0.884854, val acc: 0.896583, test acc: 0.500000, test acc (with median): 0.500000
9th epoch, train acc: 0.889146, val acc: 0.897333, test acc: 1.000000, test acc (with median): 0.750000
10th epoch, train acc: 0.891229, val acc: 0.901333, test acc: 0.

79th epoch, train acc: 0.937000, val acc: 0.935750, test acc: 1.000000, test acc (with median): 1.000000
80th epoch, train acc: 0.936250, val acc: 0.939417, test acc: 1.000000, test acc (with median): 1.000000
81th epoch, train acc: 0.936917, val acc: 0.942417, test acc: 0.750000, test acc (with median): 1.000000
82th epoch, train acc: 0.938021, val acc: 0.939583, test acc: 1.000000, test acc (with median): 1.000000
83th epoch, train acc: 0.937792, val acc: 0.938750, test acc: 1.000000, test acc (with median): 1.000000
84th epoch, train acc: 0.938542, val acc: 0.939750, test acc: 1.000000, test acc (with median): 1.000000
85th epoch, train acc: 0.936208, val acc: 0.942250, test acc: 1.000000, test acc (with median): 1.000000
86th epoch, train acc: 0.937875, val acc: 0.936167, test acc: 1.000000, test acc (with median): 1.000000
87th epoch, train acc: 0.938250, val acc: 0.935917, test acc: 1.000000, test acc (with median): 1.000000
88th epoch, train acc: 0.938396, val acc: 0.942833, tes

KeyboardInterrupt: 

In [10]:
# 1th epoch, train acc: 0.408792, val acc: 0.523917, test acc: 0.000000
# 2th epoch, train acc: 0.605125, val acc: 0.643417, test acc: 0.000000
# 3th epoch, train acc: 0.678979, val acc: 0.713083, test acc: 0.000000
# 4th epoch, train acc: 0.727250, val acc: 0.750083, test acc: 0.000000
# 5th epoch, train acc: 0.762021, val acc: 0.773583, test acc: 0.000000
# 6th epoch, train acc: 0.780417, val acc: 0.791583, test acc: 0.000000

In [11]:
# 1th epoch, train acc: 0.338646, test acc: 0.456000
# 2th epoch, train acc: 0.531000, test acc: 0.577000
# 3th epoch, train acc: 0.606146, test acc: 0.632833
# 4th epoch, train acc: 0.651458, test acc: 0.668083
# 5th epoch, train acc: 0.683938, test acc: 0.694417
# 6th epoch, train acc: 0.708542, test acc: 0.713000
# 7th epoch, train acc: 0.728292, test acc: 0.729167
# 8th epoch, train acc: 0.743021, test acc: 0.750583
# 9th epoch, train acc: 0.755292, test acc: 0.763833
# 10th epoch, train acc: 0.764208, test acc: 0.763917
# 11th epoch, train acc: 0.773229, test acc: 0.778083
# 12th epoch, train acc: 0.781708, test acc: 0.785333
# 13th epoch, train acc: 0.786958, test acc: 0.785750
# 14th epoch, train acc: 0.795396, test acc: 0.795500
# 15th epoch, train acc: 0.798271, test acc: 0.791750
# 16th epoch, train acc: 0.801896, test acc: 0.808917